# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [2]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [3]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [7]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-23'
start_date = '2016-01-01'
# Descargar el dataframe
eth_df = yf.download('ETH-USD',start_date, today).reset_index()

eth_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1835,2022-11-18,"$1,200.80","$1,226.74","$1,200.80","$1,212.30","$1,212.30",7979913378
1836,2022-11-19,"$1,212.22","$1,227.84","$1,200.48","$1,218.43","$1,218.43",5978745776
1837,2022-11-20,"$1,218.44","$1,224.08","$1,137.93","$1,142.47","$1,142.47",9651002684
1838,2022-11-21,"$1,142.40","$1,142.40","$1,084.86","$1,108.35","$1,108.35",14080099208
1839,2022-11-22,"$1,107.90","$1,136.44","$1,081.14","$1,135.17","$1,135.17",12040670755


In [19]:
# Validamos que no hay datos vacíos
eth_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-09,$308.64,$329.45,$307.06,$320.88,$320.88,893249984
1,2017-11-10,$320.67,$324.72,$294.54,$299.25,$299.25,885985984
2,2017-11-11,$298.59,$319.45,$298.19,$314.68,$314.68,842300992
3,2017-11-12,$314.69,$319.15,$298.51,$307.91,$307.91,1613479936
4,2017-11-13,$307.02,$328.42,$307.02,$316.72,$316.72,1041889984
...,...,...,...,...,...,...,...
1835,2022-11-18,"$1,200.80","$1,226.74","$1,200.80","$1,212.30","$1,212.30",7979913378
1836,2022-11-19,"$1,212.22","$1,227.84","$1,200.48","$1,218.43","$1,218.43",5978745776
1837,2022-11-20,"$1,218.44","$1,224.08","$1,137.93","$1,142.47","$1,142.47",9651002684
1838,2022-11-21,"$1,142.40","$1,142.40","$1,084.86","$1,108.35","$1,108.35",14080099208


In [25]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
eth_df = eth_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y"
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
eth_df = eth_df.rename(columns={'Date':'ds', 'Open':'y'})
eth_df

,ds,y
0,2017-11-09,$308.64
1,2017-11-10,$320.67
2,2017-11-11,$298.59
3,2017-11-12,$314.69
4,2017-11-13,$307.02
...,...,...
1835,2022-11-18,"$1,200.80"
1836,2022-11-19,"$1,212.22"
1837,2022-11-20,"$1,218.44"
1838,2022-11-21,"$1,142.40"


In [28]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
eth_df['ds'] = eth_df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1798,2022-10-12 00:00:00+00:00,"$1,279.73"
1799,2022-10-13 00:00:00+00:00,"$1,294.92"
1800,2022-10-14 00:00:00+00:00,"$1,288.05"
1801,2022-10-15 00:00:00+00:00,"$1,297.31"
1802,2022-10-16 00:00:00+00:00,"$1,275.01"


In [29]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = eth_df["ds"]
#Asigna a la variable y la columna y
y = eth_df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "grafica_eth"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
    https://facebook.github.io/prophet/docs/quick_start.html 
</p>



In [30]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet()


In [32]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(eth_df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [33]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2200,2023-11-18
2201,2023-11-19
2202,2023-11-20
2203,2023-11-21
2204,2023-11-22


In [34]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2017-11-09
1,2017-11-10
2,2017-11-11
3,2017-11-12
4,2017-11-13
...,...
2200,2023-11-18
2201,2023-11-19
2202,2023-11-20
2203,2023-11-21


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [35]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [36]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2200,2023-11-18,"$-2,157.68","$-3,193.19","$-1,100.47"
2201,2023-11-19,"$-2,169.35","$-3,173.42","$-1,152.51"
2202,2023-11-20,"$-2,183.82","$-3,231.22","$-1,166.17"
2203,2023-11-21,"$-2,198.35","$-3,303.13","$-1,137.10"
2204,2023-11-22,"$-2,213.81","$-3,278.02","$-1,246.09"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [37]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [38]:
plot_components_plotly(m, forecast)